In [0]:
#Airline data 
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
spark = SparkSession.builder.appName("Flights-Assigmnet1").getOrCreate()
#df = spark.read.csv('dbfs:/Workspace/default/airlines_flights_data_1.csv', header=True)

df = spark.read.csv("dbfs:/Volumes/workspace/Airline/air/airlines_flights_data_1.csv", header=True)

 
# Show the DataFrame

df.show()
df.count()
df.printSchema()

In [0]:
#Some data checks of the airline datasets to understand datasets
df1=df.select("*").distinct().count()
print(f"Count of r : {df1}")
# There are no duplicates in the dataset
df2=df.dropna().count()
print(f"drop values:{df2}")
#INDEX FORMAT
df2=df.dropna().count()
print(f"drop values:{df2}")


In [0]:
# # #2.Get the profiling report of the data and report if there are any data quality issues.

# import pandas as pd 
# from ydata_profiling import ProfileReport

# df_profile= pd.read_csv("/files/airlines_flights_data.csv")
# profile_report =ProfileReport(df_profile, title="Flights assigment-1 data profiling ")
# #save the file
# profile_report.to_file("/tmp/burhanuddin_profile_report.html")
# print("Report is saved successfully")

In [0]:
#3 Check if flight number is of the format <A-Z|0-9><A-Z|0-9>-<3 or 4 digits>
import re
regex= "^[A-Z0-9]{2}-\d{3,4}$"

#wanted to collect copunts of rows which are having proper flight pattern
flight_pattern= df.filter(col("flight").rlike(regex))
flight_pattern_correct=flight_pattern.count()
print(flight_pattern_correct)

#wanted to collect copunts of rows which are not having proper flight pattern
flight_pattern= df.filter(~col("flight").rlike(regex))
flight_pattern_incorrect=flight_pattern.count()
print(flight_pattern_incorrect)

#This are different airline format
airline_formats ={"6E":3,"AI":3,"UK":3,"G8":3,"SG":4,"I5":3}

def correct_flight_number(flight):
  if not flight:
    return None
  flight=str(flight).upper()

  #split airline first 2 chars and digits
  match=re.match(r"^([A-Z0-9]{2})(\d+)$",flight)
  if not match:
    return flight

  airline, number = match.groups()

  length =airline_formats.get(airline, 3)

  #zero pad number
  number =number.zfill(length)

  return f"{airline}{number}"
  
#register
correct_flight_number_udf=udf(correct_flight_number,StringType())  

#overwrite same column
df_flight=df.withColumn("flight",correct_flight_number_udf(df.flight))
df_flight.show(60,truncate=False)

In [0]:
# #3.1 Check if flight number is of the format <A-Z|0-9><A-Z|0-9>-<3 or 4 digits>
# import re
# regex= "^[A-Z0-9]{2}-\d{3,4}$"

# #wanted to collect copunts of rows which are having proper flight pattern
# flight_pattern= df.filter(col("flight").rlike(regex))
# flight_pattern_correct=flight_pattern.count()
# print(flight_pattern_correct)

# #wanted to collect copunts of rows which are not having proper flight pattern
# flight_pattern= df.filter(~col("flight").rlike(regex))
# flight_pattern_incorrect=flight_pattern.count()
# print(flight_pattern_incorrect)

# df10= df.withColumn("airline_code",regexp_extract(col("flight"),r'([A-Z0-9]{2})',1))

# df10= df10.withColumn("airline_digits",regexp_extract(col("flight"),r'(\d{3,4})',1))
# df10.show()
# df10=df10.withColumn("airline_digits",lpad(col("airline_digits"),3,'0'))
# print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

# #df10= df10.withColumn("corrected_flight_numbers",col("airline_code") + col("airline_digits"))
# df10=df10.withColumn("corrected_flight_numbers", concat_ws("",col("airline_code").cast("string"),col("airline_digits").cast("string")))
# df10.show
# df10 = df10.withColumn("flight", when(col("flight").rlike(regex),col("flight")).otherwise(col("corrected_flight_numbers"))).drop("airline_code","airline_digits","corrected_flight_numbers")
# df10.show(100)

In [0]:
#3.2 Check if the source_city & destination_city are in the master list of ‘IndianCities’ ( Delhi, Mumbai,Kolkata,Bangalore,Chennai,Hyderabad)
#how many distinct aierlines are present in dataset
df_airlines=df.select("airline").distinct()
df_airlines.show()

#source city or destination city
main_cities=["Delhi","Mumbai","Kolkata","Bangalore","Chennai","Hyderabad"]
df_source_city=df[df["source_city"].isin(main_cities)]
df_destination_city=df[df["destination_city"].isin(main_cities)]
df_other_city =df.filter(~df["source_city"].isin(main_cities) & ~df["destination_city"].isin(main_cities)) 

df_other_city.show()

print(f"Count of Other city :{df_other_city.count()}")

output_path ="/tmp/Burhanuddin_3.2_error_file.csv"
df_append_cities=df_other_city
df_append_cities.write.format("csv").mode("append").option("header","true").save(output_path)
print("Error file is saved succesfully")



In [0]:
#3.3 Check if the departure_time and arrival_time are in the master list of ‘ValidTimeslots’ (Early_Morning, Morning, Afternoon, Evening, Night, Late_Night)
valid_timeslots=["Early_Morning","Morning","Afternoon", "Evening", "Night", "Late_Night"]
df_departure_time=df[df["departure_time"].isin(valid_timeslots)]
df_arrival_time=df[df["arrival_time"].isin(valid_timeslots)]
df_other_slots =df.filter(~df["departure_time"].isin(valid_timeslots) & ~df["arrival_time"].isin(valid_timeslots)) 

df_other_slots.show()
print(f"count is :{df_other_slots.count()}")


In [0]:
#3.4 Check if the stops are in the master list of 'ValidNumberofstops' (zero, one, two_or_more)
Valid_no_of_stops=["zero","one","two_or_more"]
df_valid_stops=df.withColumn("stops", when(col("stops").isin(Valid_no_of_stops),col("stops")).otherwise(lit("zero")))
df_valid_stops.show()
print(f"the count of valuid stops is :{df_valid_stops.count()}")

In [0]:
#3.5Check if class is in the master list of ‘ValidAirlineClasses’ (Economy, Business)
df_class = df.withColumn("class",when((col("class").isNull()), "Economy")
.when((col("class")=="FirstClass") | (col("class")=="BuisnessClass"),"Buisness")
.when((col("class")=="PreiumEconomy") | (col("class")=="EconomyClass"), "Economy")
.otherwise("Economy")
)
df_class.show()
df_class.count()

In [0]:
#3.6 Check if ‘days_left’ is not 0 or negative
df_days_left= df.withColumn("days_left",when(col("days_left")<=0, lit(1)).otherwise(col("days_left")))
df_days_left.show()
print(f"the count of valuid stops is :{df_days_left.count()}")

In [0]:
#3.7 For Economy class tickets, Check if ‘price’ is not below INR1000 and not more than INR30000, (where the flight is flying to the destination directly or may take one stop.)

df_invalid_ticket_economy_pricing=df.where((col("class")=="Economy") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")<1000) | (col("price")>30000)))
df_invalid_ticket_economy_pricing.show()
print(f"the count of invalid ticket pricing is :{df_invalid_ticket_economy_pricing.count()}")
output_path ="/tmp/Burhanuddin_3.7_invalidpricing_economytickets.csv.csv"
df_append_invalidticket=df_invalid_ticket_economy_pricing
df_append_invalidticket.write.format("csv").mode("append").option("header","true").save(output_path)
print("Invalidpricing_economytickets.csv is saved succesfully")

df_economy_class_ticket=df.withColumn("price", when((col("class") == "Economy") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")<1000)),lit(1000))\
                                                   .when((col("class") == "Economy") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")>30000)),lit(30000))\
                                                   .otherwise(col("price")))
df_economy_class_ticket.show(10)  


In [0]:
#3.8 For Business class tickets, Check if ‘price’ is not below INR12000 and not more than INR105000, (where the flight is flying to the destination directly or may take one stop.)

df_invalid_ticket_buisness_pricing=df.where((col("class")=="Buisness") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")<12000) | (col("price")>105000)))
df_invalid_ticket_buisness_pricing.show(2)
print(f"the count of invalid ticket pricing is :{df_invalid_ticket_buisness_pricing.count()}")
output_path ="/tmp/Burhanuddin_3.7_invalidpricing_buisnesstickets.csv.csv"
df_append_invalidticket=df_invalid_ticket_buisness_pricing
df_append_invalidticket.write.format("csv").mode("append").option("header","true").save(output_path)
print("Invalidpricing_buisnesstickets.csv is saved succesfully")


df_buisness_class_ticket=df.withColumn("price", when((col("class") == "Buisness") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")<12000)),lit(12000))\
                                                   .when((col("class") == "Buisness") & ((col("stops")=="zero")|(col("stops")=="one")) & ((col("price")>105000)),lit(1050000))\
                                                   .otherwise(col("price")))
df_buisness_class_ticket.show(10)  


In [0]:
#4. Which airlines is running the shortest duration flight between which two cities? What’s the duration?

df_shortest_duration_run=df.select("airline","source_city","destination_city","duration")\
.groupBy("airline","source_city","destination_city").agg(min("duration").alias("shortest_duration")).orderBy("shortest_duration").limit(10)
df_shortest_duration_run.show()
df_shortest_duration_run.count()

In [0]:
#4.1 If I want to book a ticket at the best lowest price for the least travel time flight, how many days prior to the journey should I book? And at what price can I get it?

df_least_travel_time=df.select("airline","days_left","duration","price")\
                .groupBy("airline","days_left","duration").agg(min("price").alias("minimum_travel_price"))\
                .orderBy("duration")\
                .orderBy("minimum_travel_price").limit(1)
df_least_travel_time.show(5)         

In [0]:
#5.Identify the frequency of flights between different timeslots ( to know the SKY Traffic in a complete day )\


df_sky_traffic=df.filter(col("departure_time") != col("arrival_time") )\
                .select(col("departure_time").alias("Departure Timeslots"),col("arrival_time").alias("Arrival Timeslots") )\
                .groupBy("Departure Timeslots", "Arrival Timeslots")\
                .agg(count("*"))\
                .orderBy("Departure Timeslots")\
                
df_sky_traffic.show()

In [0]:
#6. a. Identify the two cities between which maximum number of flights are operated

df_max_flights=df.groupBy("source_city","destination_city").agg(count("*").alias("max_of_flights")).orderBy("max_of_flights",ascending=False)\
              .select(col("source_city"),col("destination_city"),col("max_of_flights")).limit(5)

df_max_flights.show()

  

In [0]:
#6. b. Identify the two cities between which minimum number of flights are operated


df_min_flights=df.groupBy("source_city","destination_city").agg(count("*").alias("min_of_flights")).orderBy("min_of_flights")\
              .select(col("source_city"),col("destination_city"),col("min_of_flights")).limit(5)

df_min_flights.show()

In [0]:
#7.Identify the airlines which provide the lowest price between every pair of cities in each direction for both classes

from pyspark.sql.window import Window
# df_lowest_travel_price=df.select("source_city", "destination_city","airline","class","price")\
#                 .groupBy(col("source_city").alias("From"),col("destination_city").alias("To"),"class")\
#                 .agg(min("price").alias("Best price"))\
#                 .orderBy("class",ascending=False)
# df_lowest_travel_price.count()                
# df_lowest_travel_price.show(1000)        
# df_lowest_travel_price.count()   
window_spec = Window.partitionBy("source_city", "destination_city","class").orderBy(col("price"))
df3=df.withColumn("best price",row_number().over(window_spec)).filter(col("best price") == 1).select("source_city", "destination_city","airline","class","price").distinct()
#df_lowest_travel_price.show()    
df3.show()


In [0]:
import pandas as pd
#8.
flight="AI-401"

#add new column fares 
df_fares = df.withColumn(
    "fares",
    to_json(struct(col("days_left"), col("price")))
)
#df_fares.show()
df_fares_json=df_fares.select("airline","flight","source_city","destination_city","fares")\
            .groupBy("airline","flight","source_city","destination_city")\
            .agg(collect_list("fares").alias("fares"))\
            .filter(col("flight")==flight)
df_fares_json.show(truncate=False)

output_file = '/tmp/output.jsonl'
df_fares_json.write.format("json").mode("overwrite").option("multiLine",True).save(output_file)
df5= spark.read.format("csv").load(output_file)
df5.show()

# # Write each record as a JSON object on a new line
# with open(output_file, 'w') as f:
#     for record in records:
#         f.write(json.dumps(record) + '\n')

# print(f"DataFrame successfully converted to line-by-line JSON in '{output_file}'")